In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment, Edge, Node
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from sample.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from scipy import sparse
import numpy as np
import os
from datetime import datetime, timedelta
import pandas
path_code = "/home/cbisot/pycode/"
plate_info = pandas.read_excel(path_code + 'MscThesis/plate_info/SummaryAnalizedPlates.xlsx',engine='openpyxl',header=3,)

In [2]:
def get_Pside(plate_number):
    return(plate_info.loc[plate_info['Plate #'] == plate_number]['P-bait'])

In [72]:
get_Pside(395).values[0]

'right'

In [19]:
directory = directory_project
exp = get_exp((13,116,131),directory)

begin = 2020-10-09 06:18:00 
  end = 2020-10-12 02:18:00


In [3]:
def get_raw(exp,t):
    date = exp.dates[t]
    directory_name = get_dirname(date,exp.plate)
    path_snap = exp.directory + directory_name
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    return(im)

In [4]:
def get_pos_baits(exp,t):
    raw_im = get_raw(exp,t)
    image = cv2.blur(raw_im,(100,100))
    output = image.copy()
    gray = image
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 2, 3000,minRadius = 200,maxRadius = 600,param1=10)
    # ensure at least some circles were found
    print('circles', circles)
    if circles is not None:
        # convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")
    for (x, y, r) in circles[:2]:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv2.circle(output, (x, y), r, (0, 255, 0), 4)
        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
        # show the output image
#     fig = plt.figure(figsize=(10,9))
#     ax = fig.add_subplot(111)
#     ax.imshow(output)
    return(circles[:2])

In [5]:
def get_pos_baits_aligned(exp,t):
    pos_bait = get_pos_baits(exp,t)
    date = exp.dates[t]
    directory_name = get_dirname(date, exp.plate)
    path_snap = exp.directory + directory_name
    path_tile = path_snap + "/Img/TileConfiguration.txt.registered"
    skel = read_mat(path_snap + "/Analysis/skeleton_pruned_realigned.mat")
    Rot = skel["R"]
    trans = skel["t"]
    print(Rot,trans)
    real_pos=[]
    for x,y,r in pos_bait:
        compression = 5
        xs,ys = x*compression,y*compression
        rottrans = np.dot(Rot, np.array([ys, xs])) + trans
#         rottrans = np.array([xs, ys])
        xs, ys = round(rottrans[1]), round(rottrans[0])
        real_pos.append((xs,ys))
    pos_real = {}
    if real_pos[0][0]>=real_pos[1][0]:
        pos_real['right'] = real_pos[0]
        pos_real['left'] = real_pos[1]
    else:
        pos_real['right'] = real_pos[1]
        pos_real['left'] = real_pos[0]
    return(pos_real)
        

In [24]:
plt.close('all')
t=11
baits=get_pos_baits_aligned(exp,t)

circles [[[1887.  3115.   417.6]
  [8041.  2987.   390.8]]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[ 9.99999996e-01 -9.08983427e-05]
 [ 9.08983427e-05  9.99999996e-01]] [  8.84832032 -55.68263459]


In [25]:
date = exp.dates[t]
directory_name = get_dirname(date,exp.plate)
path_snap = directory + directory_name
skel_info = read_mat(path_snap+'/Analysis/skeleton_realigned_compressed.mat')
skel = skel_info['skeleton']
#     skels.append(skel)
kernel = np.ones((5, 5), np.uint8)
itera=1
final_skel= cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera)

In [26]:
fig = plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(final_skel)
ax.scatter(baits['right'][0]//5,baits['right'][1]//5)
ax.scatter(baits['left'][0]//5,baits['left'][1]//5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
exp.ts

16

In [7]:
import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
sys.path.append( '/home/cbisot/pycode/MscThesis/sample/pipeline/functions')

from sample.notebooks.analysis.util import *
from sample.paths.directory import path_code, directory_scratch, directory_project
from sample.notebooks.analysis.data_info import *
# for treatment in ['25','baits']:
#     insts = treatments[treatment]
#     for inst in insts:
#         exp = get_exp(inst,directory_project)
#         pos_baits_time = []
#         for t in range(exp.ts):
#             baits=get_pos_baits_aligned(exp,t)
#             pos_baits_time.append(baits)
#         pickle.dump(pos_baits_time, open(f'{path_code}/MscThesis/Results/baits_{inst}.pick', "wb"))

In [32]:
bait_positions={}
for treatment in ['25','baits']:
    insts = treatments[treatment]
    for inst in insts:
        bait_positions[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/baits_{inst}.pick', "rb"))

In [63]:
bait_positions[35,0,15][0][]

{'right': (32925, 13085), 'left': (5325, 11825)}

In [ ]:
from sample.notebooks.analysis.util import *
from sample.paths.directory import path_code, directory_scratch, directory_project
from sample.notebooks.analysis.data_info import *
window=800
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        pos_baits = bait_positions[inst]
        result = get_curvature_density(inst,window,directory_project,pos_baits)
        pickle.dump(result, open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "wb"))

In [9]:
def get_pos_nut(inst, t, nutrient):
    plate_num = plate_number[inst]
    side_P = get_Pside(plate_num).values[0]
    if nutrient == 'P':
        side_nut = side_P
    else:
        side_nut ='left' if side_P=='right' else 'left'
    if side_nut == 'none':
        side_nut = 'right'
    pos_nut = bait_positions[inst][t][side_nut]
    return(pos_nut,side_nut)

In [87]:
get_pos_nut((35,0,15),0,'P')

b


((32925, 13085), 'right')

In [34]:
def get_angle(veca,vecb):
    begin = veca/np.linalg.norm(veca)
    end = vecb/np.linalg.norm(vecb)
    dot_product = min(np.dot(begin, end),1)
    if (begin[1] * end[0] - end[1] * begin[0] >= 0):  # determinant
        angle = -np.arccos(dot_product) / (2 * np.pi) * 360
    else:
        angle = np.arccos(dot_product) / (2 * np.pi) * 360
    return(angle)

def get_curvature_density_bait(window, path,pos_baits):
    column_names = ["plate","inst", "treatment", "angle", "curvature","density","growth","speed",
                    "straightness","dist_P","dist_N","angle_to_P","angle_to_N","t","hyph"]
    infos = pd.DataFrame(columns=column_names)
    for treatment in ['25','baits']:
        insts = treatments[treatment]
        for inst in insts:
            exp = get_exp(inst,path)
            skeletons = [sparse.csr_matrix(skel) for skel in exp.skeletons]
            RH, BAS, max_speeds, total_growths, widths_sp, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)
            pos_baits = bait_positions[inst]
            for hyph in RH:
                for i,t in enumerate(hyph.ts[:-1]):
                    pos_P, side_P = get_pos_nut(inst,t,'P')
                    pos_N, side_N = get_pos_nut(inst,t,'N')
                    tp1=hyph.ts[i+1]
                    segs,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
                    speed = np.sum([get_length_um(seg) for seg in segs])/get_time(exp,t,tp1)
                    total_growth = speed * get_time(exp,t,tp1)
                    curve = [pixel for seg in segs for pixel in seg]
                    pos = hyph.end.pos(t)
                    dist_P = np.linalg.norm(np.array(pos)-np.array(pos_P))
                    dist_N = np.linalg.norm(np.array(pos)-np.array(pos_N))
                    x,y = pos[0],pos[1]
                    straight_distance = np.linalg.norm(hyph.end.pos(t)-hyph.end.pos(tp1))
                    skeleton=skeletons[t][x-window:x+window,y-window:y+window]
                    density = skeleton.count_nonzero()/(window*1.725)
                    curve_array = np.array(curve)
                    res = 50
                    index = min(res,curve_array.shape[0]-1)
                    vec1 = curve_array [index]-curve_array [0]
                    vec2 = curve_array [-1]-curve_array [-index-1]
                    if np.linalg.norm(vec1)>0 and np.linalg.norm(vec2)>0:
                        angle = get_angle(vec1,vec2)
                        vec_growth =  (vec1+vec2)/2
                        vec_to_P = np.array(pos_P)-np.array(pos)
                        angle_to_P = get_angle(vec_growth,vec_to_P)
                        vec_to_N = np.array(pos_N)-np.array(pos)
                        angle_to_N = get_angle(vec_growth,vec_to_N)
                        inv_tortuosity = (straight_distance*1.725)/total_growth
                        if hyph.end.degree(tp1)<3 and inv_tortuosity>0.8 and inv_tortuosity<1.1:
                            if np.isnan((angle/total_growth)):
                                print(angle,total_growth,dot_product)
                            new_line = pd.DataFrame(
                                {   "plate": [plate_number[inst]],
                                    "inst": [inst],
                                    "treatment": [treatment],
                                    "angle": [angle],
                                    "curvature": [angle/total_growth],
                                    "density": [density],
                                    "growth": [total_growth],
                                    "speed": [speed],
                                    "straightness": [inv_tortuosity],
                                     "t": [get_time(exp,0,t)],
                                     "hyph": [hyph.end.label],
                                    "dist_P":[dist_P],
                                     "dist_N":[dist_N],
                                    "angle_to_P":[angle_to_P],
                                    "angle_to_N":[angle_to_N]
                                }
                            )  # index 0 for
                            # mothers need to be modified to resolve multi mother issue
                            infos = infos.append(new_line, ignore_index=True)
            break
        break
    return(infos)

In [35]:
infos = get_curvature_density_bait(800, directory_project,bait_positions)

begin = 2020-08-26 18:02:00 
  end = 2020-08-29 06:03:00


In [36]:
infos

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,dist_N,angle_to_P,angle_to_N,t,hyph
0,351,"(35, 0, 15)",25,-44.627955,-0.220995,1.423188,201.941059,50.485265,0.908600,30174.769676,2555.543973,-166.513522,-171.246818,0.000000,0
1,351,"(35, 0, 15)",25,18.091350,0.063647,0.925362,284.245706,71.061426,0.956046,29408.835169,3883.177307,178.189595,118.890568,0.000000,18
2,351,"(35, 0, 15)",25,59.052703,0.218164,0.765217,270.680056,67.670014,0.910669,29627.815731,3950.743221,126.875173,69.009309,4.000000,18
3,351,"(35, 0, 15)",25,-10.236040,-0.040914,0.850000,250.186905,62.546726,0.994030,29827.664022,3954.542957,171.968238,118.122538,12.000000,18
4,351,"(35, 0, 15)",25,10.731002,0.033386,0.844203,321.420439,80.355110,0.988017,29968.457318,4013.256159,159.874112,107.556851,16.000000,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,351,"(35, 0, 15)",25,16.831669,0.033944,2.695652,495.858547,123.450261,0.993517,30123.640036,5507.035954,69.279857,10.448132,52.000000,18328
198,351,"(35, 0, 15)",25,11.477627,0.024167,2.403623,474.929134,118.732283,0.989865,30008.752490,5235.354525,63.271028,4.516098,56.016667,18328
199,351,"(35, 0, 15)",25,-1.287329,-0.003983,2.012319,323.236513,80.473821,0.997782,31278.590841,15190.895464,160.987925,98.778468,52.000000,18430
200,351,"(35, 0, 15)",25,4.020104,0.010164,2.113043,395.533246,98.883311,0.999591,31436.568785,15224.193443,168.497144,106.884749,56.016667,18430


In [26]:
window = 800

In [27]:
pickle.dump(infos, open(f'{path_code}/MscThesis/Results/straight_bait_{window}_{inst}.pick', "wb"))

In [30]:
infos2 = pickle.load(open(f'{path_code}/MscThesis/Results/straight_bait_{window}_{inst}.pick', "rb"))

In [33]:
inst

(13, 116, 131)